# Multilayer perceptrons in `gluon`

Building a multilayer perceptron to classify MNIST images with `gluon` is not much harder
than [implementing softmax regression with `gluon`](../chapter02_supervised-learning/softmax-regression-gluon.ipynb), like we did in Chapter 2.
In that chapter, our entire neural network consisted 
of one Dense layer (`net = gluon.nn.Dense(num_outputs)`).

In this chapter, we're going to show you 
how to compose multiple layers together 
into a neural network.
There are two main ways to do this in Gluon and we'll walk through both. 
The first is to define a custom `Block`.
In Gluon, everything is a Block! 
Layers, losses, whole networks, they're all blocks!
So naturally, that's a flexible way to do nearly anything you want. 

We'll also make use of `gluon.nn.Sequential`.
Sequential gives us a special way of rapidly building networks
when follow a common design pattern: they look like a stack of pancakes.
Many networks follow this pattern:
a bunch of layers, one stacked on top of another,
where the output of each layer is the input to the next layer.
Sequential just takes a list of layers (we pass them in by calling `net.add(<Layer goes here!>)`.
The following unnecessary picture should give you an intuitive sense of when to (and not to) use sequential.

![](../img/sequential-not-sequential-layers.png)


## Imports
First we'll import the necessary bits.

In [1]:
from __future__ import print_function
import numpy as np
import mxnet as mx
from mxnet import nd, autograd, gluon

We'll also want to set the contexts for our data and our models.

In [2]:
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()
data_ctx = ctx
model_ctx = ctx

## The MNIST dataset

In [3]:
batch_size = 64
num_inputs = 784
num_outputs = 10
num_examples = 60000
def transform(data, label):
    return data.astype(np.float32)/255, label.astype(np.float32)
train_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=True, transform=transform),
                                      batch_size, shuffle=True)
test_data = mx.gluon.data.DataLoader(mx.gluon.data.vision.MNIST(train=False, transform=transform),
                                     batch_size, shuffle=False)

## Define the model with `gluon.Block`

Now instead of having one `gluon.nn.Dense` layer, we'll want to compose several together. 
First let's go through the most fundamental way of doing this. Then we'll introduce some shortcuts.
In `gluon` a Block has one main job - define a `forward` method that takes some NDArray input `x` and generates an NDArray output. 
Because the output and input are related to each other via NDArray operations, MXNet can take derivatives through the block automatically. 
A Block can just do something simple like apply an activation function. 
But it can also combine a bunch of other Blocks together in creative ways.
In this case, we'll just want to instantiate three Dense layers. 
The forward can then invoke the layers in turn to generate its output. 

In [4]:
class MLP(gluon.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = gluon.nn.Dense(64)
            self.dense1 = gluon.nn.Dense(64)
            self.dense2 = gluon.nn.Dense(10)
        
    def forward(self, x):
        x = nd.relu(self.dense0(x))
        x = nd.relu(self.dense1(x))
        x = self.dense2(x)
        return x

We can now instantiate a multilayer perceptron using our MLP class.
And just as with any other block, we can grab its parameters with `collect_params` and initialize them.

In [5]:
net = MLP()
net.collect_params().initialize(mx.init.Normal(sigma=.01), ctx=model_ctx)

And we can synthesize some gibberish data just to demonstrate one forward pass through the network. 

In [6]:
data = nd.ones((1,784))
net(data.as_in_context(model_ctx))


[[ 0.0010479  -0.00023263  0.00024665 -0.00137001 -0.00089217 -0.00043491
   0.00174529 -0.00114445  0.00024293 -0.0004818 ]]
<NDArray 1x10 @gpu(0)>

Because we're working with an imperative framework and not a symbolic framework, debugging Gluon Blocks is easy. If we want to see what's going on at each layer of the neural network, we can just plug in a bunch of Python print statements.

In [7]:
class MLP(gluon.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = gluon.nn.Dense(64, activation="relu")
            self.dense1 = gluon.nn.Dense(64, activation="relu")
            self.dense2 = gluon.nn.Dense(10)
        
    def forward(self, x):
        x = self.dense0(x)
        print("Hidden Representation 1: %s" % x)
        x = self.dense1(x)
        print("Hidden Representation 2: %s" % x)
        x = self.dense2(x)
        print("Network output: %s" % x)
        return x

net = MLP()
net.collect_params().initialize(mx.init.Normal(sigma=.01), ctx=model_ctx)
net(data.as_in_context(model_ctx))

Hidden Representation 1: 
[[0.         0.         0.07143721 0.         0.         0.02748545
  0.33361885 0.2867228  0.         0.         0.5456277  0.21671814
  0.20799856 0.2859339  0.         0.34327483 0.         0.4614671
  0.02448964 0.         0.01388636 0.         0.25961956 0.
  0.15838279 0.         0.10904857 0.         0.         0.02812487
  0.         0.         0.3602645  0.         0.3662219  0.
  0.36324638 0.19642079 0.35552478 0.01087478 0.6817804  0.
  0.         0.02232164 0.02738917 0.         0.         0.
  0.05049316 0.37144935 0.16642064 0.         0.07810777 0.01600346
  0.         0.         0.         0.         0.         0.
  0.         0.         0.07678016 0.6952986 ]]
<NDArray 1x64 @gpu(0)>
Hidden Representation 2: 
[[0.00646666 0.         0.01819706 0.         0.         0.00655819
  0.         0.         0.         0.         0.         0.02194543
  0.         0.         0.00290144 0.         0.         0.00796402
  0.         0.         0.00674235


[[ 0.00144656  0.00042265 -0.00119047  0.00066365  0.00035064  0.00032936
  -0.00058919 -0.00066042 -0.00112773 -0.00013794]]
<NDArray 1x10 @gpu(0)>

# Faster modeling with `gluon.nn.Sequential`

MLPs, like many deep neural networks follow a pretty boring architecture. 
Just take a list of the layers, chain them together, and return the output.
There's no reason why we have to actually define a new class every time we want to do this. 
Gluon's `Sequential` class provides a nice way of rapidly implementing this standard network architecture.
We just 

* Instantiate a Sequential (let's call it `net`) 
* Add a bunch of layers to it using `net.add(...)`

Sequential assumes that the layers arrive bottom to top (with input at the very bottom).
We could implement the same architecture as shown above using sequential in just 6 lines.

In [8]:
num_hidden = 64
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_hidden, activation="relu"))
    net.add(gluon.nn.Dense(num_outputs))

## Parameter initialization


In [9]:
net.collect_params().initialize(mx.init.Normal(sigma=.1), ctx=model_ctx)

## Softmax cross-entropy loss

In [10]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

## Optimizer

In [11]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .01})

## Evaluation metric

In [12]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for i, (data, label) in enumerate(data_iterator):
        data = data.as_in_context(model_ctx).reshape((-1, 784))
        label = label.as_in_context(model_ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

## Training loop

In [13]:
epochs = 10
smoothing_constant = .01

for e in range(epochs):
    cumulative_loss = 0
    for i, (data, label) in enumerate(train_data):
        data = data.as_in_context(model_ctx).reshape((-1, 784))
        label = label.as_in_context(model_ctx)
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(data.shape[0])
        cumulative_loss += nd.sum(loss).asscalar()


    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    print("Epoch %s. Loss: %s, Train_acc %s, Test_acc %s" %
          (e, cumulative_loss/num_examples, train_accuracy, test_accuracy))

Epoch 0. Loss: 1.166570696481069, Train_acc 0.8456166666666667, Test_acc 0.8546
Epoch 1. Loss: 0.4534545237382253, Train_acc 0.8854, Test_acc 0.8879
Epoch 2. Loss: 0.3672473588943481, Train_acc 0.90065, Test_acc 0.9047
Epoch 3. Loss: 0.32737698884010313, Train_acc 0.91115, Test_acc 0.9126
Epoch 4. Loss: 0.3004332086245219, Train_acc 0.9165666666666666, Test_acc 0.9177
Epoch 5. Loss: 0.27951788520812987, Train_acc 0.9210833333333334, Test_acc 0.9196
Epoch 6. Loss: 0.26200523080825805, Train_acc 0.92685, Test_acc 0.926
Epoch 7. Loss: 0.24663689014116924, Train_acc 0.9314166666666667, Test_acc 0.9293
Epoch 8. Loss: 0.23341047346194585, Train_acc 0.9352166666666667, Test_acc 0.9321
Epoch 9. Loss: 0.22165177921056747, Train_acc 0.9380666666666667, Test_acc 0.935


## Conclusion

In this chapter, we showed two ways to build multilayer perceptrons with Gluon. We demonstrated how to subclass `gluon.Block`, and define your own forward passes. We also showed how you might debug your network by lacing your forward pass with print statements. Finally, we showed how you could define and instantiate an equivalent network with just 6 lines of code by using `gluon.nn.Sequential`. Now that you understand the basics, you're ready to leap ahead. If you're following the book in order, then the next stop will be [dropout regularization](../chapter03_deep-neural-networks/mlp-dropout-scratch.ipynb). Other possible choices would be to start leanring about [convolutional neural networks](../chapter04_convolutional-neural-networks/cnn-scratch.ipynb) which are especialy handy for working with images, or [recurrent neural networks](../chapter05_recurrent-neural-networks/simple-rnn.ipynb), which are especially useful for natural language processing.

## Next
[Dropout regularization from scratch](../chapter03_deep-neural-networks/mlp-dropout-scratch.ipynb)

For whinges or inquiries, [open an issue on  GitHub.](https://github.com/zackchase/mxnet-the-straight-dope)